In [1]:
#Bibliotecas do agente 
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, AgentType, AgentExecutor # Usado para os sub-agentes
from langchain_core.tools import StructuredTool # Para ferramentas com schema
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory # Importação da memória / definição de contexto para armazenamento da memória
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder # Para o agente principal
from pydantic import BaseModel, Field # Necessário para StructuredTool
from langchain_core.messages import HumanMessage, AIMessage 


#Funções diversas em Python
import os
import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt
import re

#importações para RAG
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma #banco vetorial
import shutil 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings




In [3]:
#1) Criação da pasta onde estão as informações
docs_folder = "docs"
csv_file_path = os.path.join(docs_folder, "dataset.csv")

#carregar os documentos
loader = TextLoader(csv_file_path, encoding="utf-8")
documents = loader.load()
print(f"DEBUG: Documentos carregados. Número de documentos: {len(documents)}")

# Dividir documentos em chunks (pedaços menores)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
print(f"DEBUG: Número de chunks criados: {len(splits)}")
print(f"DEBUG: Primeiro chunk: {splits[0].page_content[:100]}...")

embeddings_model = 'nomic-embed-text'
embeddings = OllamaEmbeddings(model=embeddings_model)

DEBUG: Documentos carregados. Número de documentos: 1
DEBUG: Número de chunks criados: 74
DEBUG: Primeiro chunk: DATA,Marca,APRESENTACAO,EAN,MDTR_UND,CPV_UND
2024-08-01,LUTEAL (A4H),LUTEAL CAP MOLE BLX60,789665804...


In [4]:
# 2) Criação ou Carregamento do banco vetorial em chroma_db
# Definindo um caminho de pasta específico para este projeto
chroma_db_path = "./chroma_db_agente_local" 
vectorstore = None # Inicializa o vectorstore como None

if os.path.exists(chroma_db_path):
    print(f"DEBUG: Removendo diretório existente do ChromaDB para forçar atualização: '{chroma_db_path}'")
    shutil.rmtree(chroma_db_path)
    print("DEBUG: Diretório do ChromaDB removido com sucesso.")

print("DEBUG: Criando e preenchendo novo ChromaDB com documentos atualizados...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=chroma_db_path
)
print("DEBUG: Novo ChromaDB criado e preenchido com sucesso com dados atualizados!")

# teste. Contagem de Documentos
print(f"\nDEBUG: Número de chunks originais (splits): {len(splits)}")
try:
    chroma_collection = vectorstore._collection
    count_in_db = chroma_collection.count()
    print(f"DEBUG: Número de documentos no ChromaDB: {count_in_db}")

    if count_in_db == len(splits):
        print("DEBUG: A contagem de documentos no ChromaDB corresponde ao número de splits.")
    else:
        print("ALERTA: A contagem de documentos no ChromaDB NÃO corresponde ao número de splits. Pode haver um problema.")
except Exception as e:
    print(f"ERRO: Não foi possível contar documentos no ChromaDB: {e}")

DEBUG: Removendo diretório existente do ChromaDB para forçar atualização: './chroma_db_agente_local'
DEBUG: Diretório do ChromaDB removido com sucesso.
DEBUG: Criando e preenchendo novo ChromaDB com documentos atualizados...
DEBUG: Novo ChromaDB criado e preenchido com sucesso com dados atualizados!

DEBUG: Número de chunks originais (splits): 74
DEBUG: Número de documentos no ChromaDB: 74
DEBUG: A contagem de documentos no ChromaDB corresponde ao número de splits.


In [5]:
#3) Criação de funções

#criação do dataset
df_info_dataset = None
try:
    df_info_dataset = pd.read_csv(csv_file_path)
    print(f"DEBUG: Dataset CSV carregado em DataFrame. Linhas: {len(df_info_dataset)}")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{csv_file_path}' não foi encontrado.")
except Exception as e:
    print(f"ERRO: Não foi possível carregar o CSV para DataFrame: {e}")


def buscar_documentos (query: str) -> str:
    """Use esta ferramenta para **buscar informações textuais relevantes** do dataset armazenado no chroma_db_agente_local.
    Esta ferramenta aceita uma 'query' em **linguagem natural** que descreve a informação que você precisa,
    como 'detalhes sobre o produto X' ou 'o que é mencionado sobre a Marca Y'.
    Ela **retorna o conteúdo relevante dos documentos em formato de texto**.
    **Não a utilize para obter dados numéricos ou tabulares para cálculos ou agregações.**
    Exemplo de uso: buscar_documentos(query="descrição da marca LUTEAL")
    """
    
    
    retriever = vectorstore.as_retriever()
    docs = retriever.invoke(query)
    if docs:
        return "\n\n".join([doc.page_content for doc in docs])
    else:
        return "Não encontrei informações relevantes nos documentos para esta consulta."
    
def sql_documentos (query_sql: str) -> str:
    """Use esta ferramenta para **consultas quantitativas/qualitativas** dentro do dataset chamado 'df_info_dataset'
    Esta ferramenta exige uma query onde o 'from' deve ser sempre 'df_info_dataset
    exemplo1 soma: 
    select max(Data), Marca, sum(CPV_UND) 
    from df_info_dataset
    group by max(Data), Marca

    exemplo2 conversão:
    select sum(CPV_UND) / sum(MDTR_UND)
    from df_info_dataset
    group by Marca

    exemplo3 conversão:
    select sum(CPV_UND) / sum(MDTR_UND)
    from df_info_dataset
    where Marca = 'MOTORE (A4H)'
    group by Marca
    """
    
    global df_info_dataset # Permite que a função acesse o df_data global
    if df_info_dataset is None:
        return "Erro: O DataFrame 'df_info_dataset' não foi carregado. Não é possível executar consultas SQL."

    colunas_originais = df_info_dataset.columns.tolist()
    
    # Cria dicionário de mapeamento simples
    mapeamento = {}
    for col in colunas_originais:
        col_nova = col.upper()
        col_nova = re.sub(r"[^A-Z0-9]", "_", col_nova)  # Substitui espaços/símbolos por _
        col_nova = re.sub(r"_+", "_", col_nova).strip("_")  # Remove múltiplos _
        mapeamento[col] = col_nova

    # Aplica o mapeamento
    df_info_dataset.columns = [mapeamento[col] for col in df_info_dataset.columns]
    

    result = sqldf(query_sql, {'df_info_dataset': df_info_dataset}) # Mapeamento do dataset para o agente

   
    if not result.empty and len(result.columns) == 1 and len(result) == 1:
        return str(result.iloc[0, 0]) # Retorna o valor da única célula como string
    else:
        return result.to_string(index=False)


DEBUG: Dataset CSV carregado em DataFrame. Linhas: 765


In [6]:
#4) Criação de Schema
class buscar_documentos_input(BaseModel):
    query: str = Field(description="Consulta na documentaçao.")

class sql_documentos_input(BaseModel):
    query_sql: str = Field(description="Consulta SQL no dataset df_info_dataset")

In [7]:
#5) Criação das ferramentas estruturadas
buscar_tool = StructuredTool.from_function(
    func=buscar_documentos,
    name='buscar_documentos',
    description=buscar_documentos.__doc__.strip(),
    args_schema=buscar_documentos_input
)

query_tool = StructuredTool.from_function(
    func=sql_documentos,
    name='sql_documentos',
    description=sql_documentos.__doc__.strip(),
    args_schema=sql_documentos_input
)

In [8]:
#6) Criação da memória dos agentes
main_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=5 # Mantenha as últimas 5 interações (5 pares de HumanMessage/AIMessage)
)

C:\Users\Pichau\AppData\Local\Temp\ipykernel_14664\2064540434.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  main_memory = ConversationBufferWindowMemory(


In [ ]:
#7) Criação do Agente
llm = OllamaLLM(model="qwen3",
                temperature=0.2,
                max_tokens=500              
                )

agent_executor = initialize_agent(
    tools=[query_tool, buscar_tool],
    llm=llm,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=main_memory, # Adicionando memória
    agent_kwargs={"system_message": """Você é um analista de dados focado em trazer os resultados de projetos.
    Sempre utilize as ferramentas disponíveis para responder às perguntas do usuário.

    **IMPORTANTE: Seu processo de pensamento deve seguir o formato ReAct (Thought, Action, Action Input, Observation, Thought, Final Answer).**

    **Regras de Uso das Ferramentas:**
    1.  **Para perguntas de caráter quantitativo ou qualitativo que exigem análise de dados tabulares, agregação (COUNT, SUM, AVG), filtragem precisa ou qualquer cálculo sobre o dataset, VOCÊ DEVE usar a ferramenta 'query_tool'. Lembre-se que o nome da tabela no SQL é 'df_info_dataset'.**
        * Exemplo de Thought para esta ferramenta: "A pergunta é quantitativa, preciso usar query_tool para contar as marcas."
        * Exemplo de Final Answer após usar esta ferramenta: "Existem X marcas no dataset."
        * **EXEMPLO DE USO PARA CÁLCULO DE CONVERSÃO (use este padrão!):** Para "Me diga a conversão da Marca Artrolive em maio/2024", a query SQL seria:
          `SELECT SUM(CPV_UND) * 1.0 / SUM(MDTR_UND) FROM df_info_dataset WHERE Marca = 'ARTROLIVE (A4H)' AND strftime('%Y-%m', DATA) = '2024-05'`
          **Lembre-se de converter para percentual no Final Answer.**
                  
    2.  **Para perguntas sobre informações descritivas que precisam de uma interpretação de texto, ou para buscar trechos de texto específicos em documentos, use a ferramenta 'buscar_tool'.**
        * Exemplo de Thought para esta ferramenta: "A pergunta é descritiva, preciso buscar informações no texto dos documentos."

    **Estrutura do Dataset 'df_info_dataset' (Colunas e Tipos Principais):**
    * `DATA`: Tipo data (yyyy-mm-dd).
    * `MARCA`: Tipo texto (ex: 'LUTEAL (A4H)').
    * `APRESENTACAO`: Tipo texto (ex: 'LUTEAL CAP MOLE BLX60').
    * `EAN`: Tipo texto/numérico (código, ex: '7896658040894').
    * `MDTR_UND`: Tipo numérico (Unidades de MDTR).
    * `CPV_UND`: Tipo numérico (Unidades de CPV).

    **Sinônimos e Definições Adicionais:**
    * Marca pode ser também 'Produto'.
    * Data pode ser também 'Período, Data ou Ocorrência'.
    * Apresentacao pode ser também 'SKU'.
    * EAN pode ser também 'Código'.
    * CPV_UND pode ser também 'UNIDADES CPV','CPV_UNIDADES','UND_CAMPANHA','UNIDADES CAMPANHA'.
    * Conversão é o cálculo: `sum(CPV_UND) / sum(MDTR_UND)`. O resultado é em percentual.
    * Caso o usuário já traga o nome da Marca, não se esqueça de adicionar o sulfixo ' (A4H)' caso ele não tenha escrito.

    * Se uma pergunta não se encaixar em nenhuma ferramenta, responda diretamente.
    * **Sempre responda de forma resumida e coerente ao que foi perguntado, focando na resposta final APÓS usar e interpretar o resultado da ferramenta.**
    """
}

)
print("DEBUG: Agente configurado com sucesso para LLM local.")

DEBUG: Agente configurado com sucesso para LLM local.


C:\Users\Pichau\AppData\Local\Temp\ipykernel_14664\3880011911.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [10]:
# 8) Testando o agente
chat_history = []

print("\n--- Teste 1: Pergunta 1 ---")
user_input_1 = "Quantas Marcas existem neste arquivo?"
response_1 = agent_executor.invoke({"input": user_input_1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_1), AIMessage(content=response_1["output"])])
print(f"Resposta: {response_1['output']}\n")



--- Teste 1: Pergunta 1 ---


> Entering new AgentExecutor chain...
<think>
Okay, the user is asking how many brands exist in the file. The dataset is called 'df_info_dataset'. To find the number of unique brands, I should use the sql_documentos tool. The SQL query needs to count distinct entries in the 'Marca' column. Let me check the syntax. The correct query would be SELECT COUNT(DISTINCT Marca) FROM df_info_dataset. That should give the number of unique brands. I don't need to use the other tool since it's a quantitative question. Let me execute that.
</think>

Thought: Preciso determinar o número de marcas únicas no dataset. Para isso, devo usar a ferramenta sql_documentos com uma consulta que conte as marcas distintas.

Action: sql_documentos
Action Input: SELECT COUNT(DISTINCT Marca) FROM df_info_dataset

Observation: 35
Thought:<think>
Okay, the user asked how many brands exist in the file. I used the sql_documentos tool to run the query SELECT COUNT(DISTINCT Marca) FROM df_in

In [11]:
print("\n--- Teste 2: Pergunta de cálculo ---")
user_input_2 = "Considerando o total de unidades CPV, qual foi a marca que mais vendeu?"
response_2 = agent_executor.invoke({"input": user_input_2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_2), AIMessage(content=response_2["output"])])
print(f"Resposta: {response_2['output']}\n")



--- Teste 2: Pergunta de cálculo ---


> Entering new AgentExecutor chain...
<think>
Okay, let's tackle this question. The user wants to know which brand had the highest sales in terms of total CPV units. 

First, I need to understand what data I have. The dataset is called df_info_dataset, and it includes columns like CPV_UND, which I assume represents the number of units sold for each brand. The goal is to find the brand with the maximum sum of CPV_UND.

Since the question is about aggregating numerical data (summing CPV_UND per brand and finding the maximum), the sql_documentos tool is the right choice here. The tool requires a SQL query that groups by brand and sums CPV_UND, then selects the brand with the highest value.

I should structure the SQL query to group by the brand column (probably named 'Marca') and calculate the sum of CPV_UND for each. Then, order the results in descending order and limit to the top result. That should give the brand with the highest total units.

Wa

In [12]:
print("\n--- Teste 3: Pergunta sobre documentos ---")
user_input_3 = "Qual a terceira maior marca vendido em maio de 2024?"
response_3 = agent_executor.invoke({"input": user_input_3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_3), AIMessage(content=response_3["output"])])
print(f"Resposta: {response_3['output']}\n")



--- Teste 3: Pergunta sobre documentos ---


> Entering new AgentExecutor chain...
<think>
Okay, I need to find the third largest brand sold in May 2024. Let me start by figuring out how to approach this.

First, I should check the dataset 'df_info_dataset' for sales data. The user mentioned using sql_documentos for quantitative queries. So, I need to write an SQL query that sums the sales (probably CPV_UND) for each brand in May 2024. Then, order them in descending order and pick the third one.

Wait, the SQL function requires the 'from' clause to be 'df_info_dataset'. Also, the example uses grouping with max(Data), but maybe I should just group by Marca and sum CPV_UND. Let me structure the query correctly.

I need to filter the data for May 2024. Assuming the 'Data' column is a date, I should use a WHERE clause to select records where Data is between '2024-05-01' and '2024-05-31'. Then group by Marca, sum CPV_UND, order by the sum in descending order, and limit to the top 3 to find

In [15]:
print("\n--- Teste 4: Outra pergunta sobre documentos ---")
user_input_4 = "Me diga a conversão da Marca Motore em junho de 2024"
response_4 = agent_executor.invoke({"input": user_input_4, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_4), AIMessage(content=response_4["output"])])
print(f"Resposta: {response_4['output']}\n")


--- Teste 4: Outra pergunta sobre documentos ---


> Entering new AgentExecutor chain...
<think>
Okay, the user is asking for the conversion of the "Motore" brand in June 2024. Let me break this down. First, I need to understand what "conversão" means here. In the context of the provided tools, the examples show that conversion refers to the ratio of CPV_UND to MDTR_UND. So, the user wants the sum of CPV_UND divided by the sum of MDTR_UND for the Motore brand in June 2024.

I should use the sql_documentos tool since it's a quantitative calculation. The query needs to select the sum of CPV_UND divided by sum of MDTR_UND, filtered by Marca = 'MOTORE (A4H)' and Data in June 2024. Wait, the example uses 'group by Marca', but since we're already filtering by Marca, maybe the group by isn't necessary. But to be safe, perhaps include it. Also, the date format in the dataset might be important. The example uses 'max(Data)', but the user specified June 2024. So the where clause should specify 